# one notebook

notebooks are literate programs encoded in json. as literate programs, they concurrently represent both code and formal documentation. a single notebook can be through of as a program, a test, or documentation.

`dgaf` configures complete development environments for notebooks.

> one blog post.

In [1]:
import pytest
@pytest.fixture
def project(pytester):
    import nbformat, dgaf
    name = "sample_notebook_project"
    pytester.makefile(".ipynb", **{name: nbformat.v4.writes(
        nbformat.v4.new_notebook(cells=[
            nbformat.v4.new_markdown_cell("a sample notebook for testing."),
            nbformat.v4.new_code_cell("import pandas"),
        ])
    )})
    project = dgaf.Project(pytester.path)
    yield project

In [2]:
def test_cli(pytester, project):
    configure = pytester.run(*"python -m dgaf configure --no-venv".split())
    assert not configure.ret
    assert not pytester.run(*"python -m pip install . --no-deps".split()).ret
    import importlib_metadata
    assert importlib_metadata.distribution("sample_notebook_project")
    with __import__('importnb').Notebook():
        import sample_notebook_project

In [3]:
def test_project(project):
    import dgaf
    assert project.fs.files
    assert project.get_name() == "sample_notebook_project"
    assert "nbval" in project.get_test_requires()
    assert not project.get_url()
    assert project.get_requires() == ["pandas"]

In [4]:
def test_flit(pytester, project):
    # when there is no url we can initialize one in a few ways.
    import dgaf, shutil
    pyproject = project/dgaf.PYPROJECT_TOML
    assert not pyproject.exists()
    project.to_flit()
    assert pyproject.exists()
    assert not pytester.run(*"flit build".split()).ret

In [5]:
def test_setuptools(pytester, project):
    import dgaf
    setup = project/dgaf.SETUP_CFG
    assert not setup.exists()
    project.to_setuptools()
    assert setup.exists()
    project.to_setup_py()
    assert not (pytester.run(*"python setup.py build --dry-run".split())).ret

## a blog post

a useful pattern for developing blog content is to decouple the posts as gist, then reference them in a macro repository that handles general publishing. each blog post(s), as small units of though in `gist`, make them postable as gist.

A single notebook, markdown file, or rst document may represent a post in the file name conforms to the `"YYYY-MM-DD-title.ext"` pattern.

In [6]:
import pytest
@pytest.fixture
def post(pytester):
    import nbformat, dgaf
    name = "2020-12-25-a-notebook-post"
    pytester.makefile(".ipynb", **{name: nbformat.v4.writes(
        nbformat.v4.new_notebook(cells=[
            nbformat.v4.new_markdown_cell("a sample notebook for testing."),
            nbformat.v4.new_code_cell("import pandas"),
        ])
    )})
    yield dgaf.Project(pytester.path)

In [7]:
def test_post(pytester, post):
    import dgaf
    name = post.get_name()
    assert name == "a_notebook_post"
    post.to_flit()
    assert not pytester.run(*"flit build".split()).ret